ASSIGNMENT3


## 1 Downloading the **Dakshina** dataset:


In [1]:
!pip install wandb==0.12.2

## 2 Processing the dataset
### 2.1 Data Processing:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
import cv2


class DataProcessing():

    def __init__(self, path, s_lang = 'en', t_lang = "te"):
    
        self.s_lang = s_lang
        self.t_lang = t_lang
    
        self.trainpath = os.path.join(path, t_lang, "lexicons", t_lang+".translit.sampled.train.tsv")
        self.valpath = os.path.join(path, t_lang, "lexicons", t_lang+".translit.sampled.dev.tsv")
        self.testpath = os.path.join(path, t_lang, "lexicons", t_lang+".translit.sampled.test.tsv")

        self.train = pd.read_csv(
            self.trainpath,
            sep="\t",
            names=["tgt", "src", "count"],
        )
        self.test = pd.read_csv(
            self.testpath,
            sep="\t",
            names=["tgt", "src", "count"],
        )
        self.val = pd.read_csv(
            self.valpath,
            sep="\t",
            names=["tgt", "src", "count"],
        )

        #Train Data
        self.train_data = self.preprocess(self.train["src"].to_list(), self.train["tgt"].to_list())
        (
            self.train_encoder_input,
            self.train_decoder_input,
            self.train_decoder_target,
            self.source_voccab,
            self.target_voccab,
        ) = self.train_data

        #character to integer and integer to character
        self.src_charTOint, self.src_intTOchar = self.source_voccab
        self.tar_charTOint, self.tar_intTOchar = self.target_voccab

        #Validation Data
        self.val_data = self.encode(
            self.val["src"].to_list(),
            self.val["tgt"].to_list(),
            list(self.src_charTOint.keys()),
            list(self.tar_charTOint.keys()),
            src_charTOint=self.src_charTOint,
            tar_charTOint=self.tar_charTOint,
        )
        self.val_encoder_input, self.val_decoder_input, self.val_decoder_target = self.val_data
        self.src_charTOint, self.src_intTOchar = self.source_voccab
        self.tar_charTOint, self.tar_intTOchar = self.target_voccab

        #Test Data
        self.test_data = self.encode(
            self.test["src"].to_list(),
            self.test["tgt"].to_list(),
            list(self.src_charTOint.keys()),
            list(self.tar_charTOint.keys()),
            src_charTOint=self.src_charTOint,
            tar_charTOint=self.tar_charTOint,
        )
        self.test_encoder_input, self.test_decoder_input, self.test_decoder_target = self.test_data
        self.src_charTOint, self.src_intTOchar = self.source_voccab
        self.tar_charTOint, self.tar_intTOchar = self.target_voccab


    def dictionary_lookup(self, voccab):
        charTOint = dict([(char, i) for i, char in enumerate(voccab)])
        intTOchar = dict((i, char) for char, i in charTOint.items())
        return charTOint, intTOchar

    def preprocess(self, source , target):
        source_chars = set()
        target_chars = set()

        source = [str(x) for x in source]
        target = [str(x) for x in target]

        source_words,target_words = [],[]
        for src, tgt in zip(source, target):
            tgt = "\t" + tgt + "\n"
            
            source_words.append(src)
            target_words.append(tgt)

            for char in src:
                if char not in source_chars:
                    source_chars.add(char)

            for char in tgt:
                if char not in target_chars:
                    target_chars.add(char)

        source_chars = sorted(list(source_chars))
        target_chars = sorted(list(target_chars))

        #Add space
        source_chars.append(" ")
        target_chars.append(" ")

        num_encoder_tokens = len(source_chars)
        num_decoder_tokens = len(target_chars)
        max_source_length = max([len(txt) for txt in source_words])
        max_target_length = max([len(txt) for txt in target_words])

        print("No. of samples:", len(source))
        print("Src voccab length:", num_encoder_tokens)
        print("Tar voccab length:", num_decoder_tokens)
        print("Max iput sequence length:", max_source_length)
        print("Max output sequence length:", max_target_length)

        return self.encode(source_words, target_words, source_chars, target_chars)
    
    def encode(self, source, target, source_chars, target_chars, src_charTOint=None, tar_charTOint=None):
        num_decoder_tokens = len(target_chars)
        num_encoder_tokens = len(source_chars)
        max_source_length = max([len(txt) for txt in source])
        max_target_length = max([len(txt) for txt in target])

        source_voccab, target_voccab = None, None
        if src_charTOint == None and tar_charTOint == None:

            print("Dictionary lookups for char to int mapping and vice versa")
            src_charTOint, src_intTOchar = self.dictionary_lookup(source_chars)
            tar_charTOint, tar_intTOchar = self.dictionary_lookup(target_chars)

            source_voccab = (src_charTOint, src_intTOchar)
            target_voccab = (tar_charTOint, tar_intTOchar)

        encoder_input_data = np.zeros(
            (len(source), max_source_length, num_encoder_tokens), dtype="float32"
        )
        decoder_input_data = np.zeros(
            (len(source), max_target_length, num_decoder_tokens), dtype="float32"
        )
        decoder_target_data = np.zeros(
            (len(source), max_target_length, num_decoder_tokens), dtype="float32"
        )

        for i, (input_text, target_text) in enumerate(zip(source, target)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, src_charTOint[char]] = 1.0
            encoder_input_data[i, t + 1 :, src_charTOint[" "]] = 1.0
            for t, char in enumerate(target_text):
                
                decoder_input_data[i, t, tar_charTOint[char]] = 1.0
                if t > 0:
                    decoder_target_data[i, t - 1, tar_charTOint[char]] = 1.0

            decoder_input_data[i, t + 1 :, tar_charTOint[" "]] = 1.0
            decoder_target_data[i, t:, tar_charTOint[" "]] = 1.0

        if source_voccab != None and target_voccab != None:
            return (
                encoder_input_data,
                decoder_input_data,
                decoder_target_data,
                source_voccab,
                target_voccab,
            )
        else:

            return encoder_input_data, decoder_input_data, decoder_target_data


### 2.2 Processing the database

Default input language is English and output language is Telugu

In [2]:
import numpy as np
import pandas as pd
import os

DATAPATH = r"dakshina_dataset_v1.0"

dataBase = DataProcessing(DATAPATH) 



No. of samples: 58550
Src voccab length: 27
Tar voccab length: 66
Max iput sequence length: 25
Max output sequence length: 22
Dictionary lookups for char to int mapping and vice versa


## 3. RNN model for seq2seq machine translation 
### 3.1 Seq2Seq Translation Model class

In [4]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
 

#from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model, Sequential,  Model
from tensorflow.keras.callbacks import EarlyStopping



class S2STranslation():

    def __init__(self, modelConfigDict, srcChar2Int, tgtChar2Int, using_pretrained_model = False):
        self.nEncoder = modelConfigDict["nEncoder"]
        self.cell_type = modelConfigDict["cell_type"]
        self.latentDim = modelConfigDict["latentDim"]
        self.dropout = modelConfigDict["dropout"]
        self.nDecoders = modelConfigDict["nDecoders"]
        self.hidden = modelConfigDict["hidden"]
        self.tgtChar2Int = tgtChar2Int
        self.srcChar2Int = srcChar2Int

    def build_configurable_model(self):   

        #RNN    
        if self.cell_type == "RNN":
            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.nEncoder + 1):
                encoder = SimpleRNN(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state = encoder(encoder_inputs)
            encoder_states = [state]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.nDecoders + 1):
                decoder = SimpleRNN(
                    self.latentDim,
                    return_sequences=True,
                    return_state=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model

        #LSTM
        elif self.cell_type == "LSTM":

            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.nEncoder + 1):
                encoder = LSTM(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state_h, state_c = encoder(encoder_outputs)
            encoder_states = [state_h, state_c]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.nDecoders + 1):
                decoder = LSTM(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _, _ = decoder(
                    decoder_outputs, initial_state=encoder_states
                )

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model
        #GRU
        elif self.cell_type == "GRU":

            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.nEncoder + 1):
                encoder = GRU(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state = encoder(encoder_inputs)
            encoder_states = [state]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.nDecoders + 1):
                decoder = GRU(
                    self.latentDim,
                    return_sequences=True,
                    return_state=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model

### 3.2 Model Training


In [5]:
import wandb
import numpy as np
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.layers import RNN, LSTM, GRU, Dense
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from wandb.keras import WandbCallback

#using a gpu
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


def train():
    default_config = {
        "cell_type": "RNN",
        "latentDim": 256,
        "hidden": 128,
        "optimiser": "rmsprop",
        "numEncoders": 1,
        "numDecoders": 1,
        "dropout": 0.2,
        "epochs": 1,
        "batch_size": 64,
    }

    wandb.init(config=default_config)
    wandb.init(config=default_config,  project="Assignment-3_WithoutAttention", entity="karanwxlia")
    config = wandb.config
    
    wandb.run.name = (
        str(config.cell_type)
        + dataBase.source_lang
        + str(config.numEncoders)
        + "_"
        + dataBase.target_lang
        + "_"
        + str(config.numDecoders)
        + "_"
        + config.optimiser
        + "_"
        + str(config.epochs)
        + "_"
        + str(config.dropout) 
        + "_"
        + str(config.batch_size)
        + "_"
        + str(config.latentDim)
    )
    wandb.run.save()

    modelInit = S2STranslation(config,srcChar2Int=dataBase.src_charTOint, tgtChar2Int=dataBase.tar_charTOint)
    
    model = modelInit.build_attention_model()
    model.summary()
    model.compile(
        optimizer=config.optimiser,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    earlystopping = EarlyStopping(
        monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
    )
    model.fit(
        [dataBase.train_encoder_input, dataBase.train_decoder_input],
        dataBase.train_decoder_target,
        batch_size=config.batch_size,
        epochs=config.epochs,
        validation_data=([dataBase.val_encoder_input, dataBase.val_decoder_input], dataBase.val_decoder_target),
        callbacks=[earlystopping, WandbCallback()],
    )

    model.save(os.path.join("./TrainedAttentionModels", wandb.run.name))    
    wandb.finish()
    
    #return model






Running the wandb sweep: 

In [7]:
  
sweep_config = {
    "name": "Bayesian Sweep without attention",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},
        
        "latentDim": {"values": [256]},
        
        "hidden": {"values": [128, 64]},
        
        "optimiser": {"values": ["rmsprop", "adam"]},
        
        "numEncoders": {"values": [1, 2, 3]},
        
        "numDecoders": {"values": [1, 2, 3]},
        
        "dropout": {"values": [0.1, 0.2, 0.3]},
        
        "epochs": {"values": [5,10,15, 20]},
        
        "batch_size": {"values": [32, 64]},
    },
}

sweep_id = wandb.sweep(sweep_config, project="Assignment-3_WithoutAttention", entity="karanwxlia")

wandb.agent(sweep_id, train, count = 200)


#train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vn4o92i4
Sweep URL: https://wandb.ai/karanwxlia/Assignment-3_WithoutAttention/sweeps/vn4o92i4


wandb: Agent Starting Run: xsqmjfi5 with config:
wandb: 	batch_size: 64
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	hidden: 128
wandb: 	latentDim: 256
wandb: 	numDecoders: 1
wandb: 	numEncoders: 2
wandb: 	optimiser: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karanwxlia (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep
wandb: wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, None, 256),  290816      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 input_2 (InputLayer)           [(None, None, 66)]   0           []                               
                                                                                              

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
915/915 [==============================] - ETA: 0s - loss: 1.1653 - accuracy: 0.6827WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000002C54FB95F30> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
915/915 [==============================] - 114s 120ms/step - loss: 1.1653 - accuracy: 0.6827 - val_loss: 1.5201 - val_accuracy: 0.6961
Epoch 2/20
915/915 [==============================] - 107s 117ms/step - loss: 0.8790 - accuracy: 0.7496 - val_loss: 1.7053 - val_accuracy: 0.7126
Epoch 3/20
915/915 [==============================] - 103s 112ms/step - loss: 0.6962 - accuracy: 0.7988 - val_loss: 1.6767 - val_accuracy: 0.7396
Epoch 4/20
915/915 [==============================] - 122s 133ms/step - loss: 0.5392 - accuracy: 0.8409 - val_loss: 1.7534 - val_accuracy: 0.7610
Epoch 5/20
915/915 [==============================] - 132s 145ms/step - loss: 0.4472

INFO:tensorflow:Assets written to: ./TrainedModels\LSTMen1_te_2_rmsprop_20_0.2_64_256\assets


INFO:tensorflow:Assets written to: ./TrainedModels\LSTMen1_te_2_rmsprop_20_0.2_64_256\assets
